<h1>Java 8 Java 8 Java 8 Java 8 Java 8</h1>

In [1]:
System.out.println("Hello stdout!");
System.err.println("Hello stderr!");

"Hello Out!";

Hello stdout!


Hello stderr!


Hello Out!

In [2]:
import java.util.function.*;
import java.util.stream.*;

# 1. Interfaces

## Typical Java 7 Interface

In [3]:
interface Calculator {
    int add(int first, int second);
    int subtract(int first, int second);
    int divide(int number, int divisor);
    int multiply(int first, int second);
}

In [4]:
class BasicCalculator implements Calculator {

    @Override
    public int add(int first, int second) {
        return first + second;
    }

    @Override
    public int subtract(int first, int second) {
        return first - second;
    }

    @Override
    public int divide(int number, int divisor) {
        if (divisor == 0) {
            throw new IllegalArgumentException("El divisor no puede ser cero.");
        }
        return number / divisor;
    }

    @Override
    public int multiply(int first, int second) {
        return first * second;
    }
}

In [5]:
Calculator calculator = new BasicCalculator();
int sum = calculator.add(1, 2);
sum;

3

In [6]:
calculator

REPL.$JShell$18$BasicCalculator@64533006

In [7]:
BasicCalculator basicCalculator = new BasicCalculator();
int difference = basicCalculator.subtract(3, 2);
difference;

1

In [8]:
basicCalculator

REPL.$JShell$18$BasicCalculator@6f334d51

In [9]:
if (calculator instanceof Calculator)
    System.out.println("Calculator");
if (calculator instanceof BasicCalculator)
    System.out.print("BasicCalculator");

Calculator
BasicCalculator

## Factory

In [10]:
public abstract class CalculatorFactory {

    public static Calculator getInstance() {
        return new BasicCalculator();
    }
}

In [11]:
Calculator calculator = CalculatorFactory.getInstance()

In [12]:
int sum = calculator.add(1, 2);
sum;

3

## Java 8: Static methods inside interfaces

In [13]:
public interface Calculator {

    static Calculator getInstance() {
        return new BasicCalculator();
    }
    
    int add(int first, int second);
    int subtract(int first, int second);
    int divide(int number, int divisor);
    int multiply(int first, int second);
}

In [14]:
Calculator calculator = Calculator.getInstance();
calculator;

REPL.$JShell$18C$BasicCalculator@7dcc57da

In [15]:
if (calculator instanceof Calculator)
    System.out.println("Calculator");
if (calculator instanceof BasicCalculator)
    System.out.print("BasicCalculator");

Calculator
BasicCalculator

This doesn't work without casting:

In [16]:
BasicCalculator basicCalculator = calculator

CompilationException: 

In [17]:
BasicCalculator basicCalculator = (BasicCalculator) calculator

## Java 8: Default methods in interfaces

In [18]:
public interface Calculator {

    static Calculator getInstance() {
        return new BasicCalculator();
    }
    
    int add(int first, int second);
    int subtract(int first, int second);
    int divide(int number, int divisor);
    int multiply(int first, int second);
    
    default int remainder(int number, int divisor) {
        return subtract(number, multiply(divisor, divide(number, divisor)));
    }
}

In [19]:
Calculator calculator = Calculator.getInstance();
calculator.remainder(7, 2)

1

This default method can be reimplemented in classes implementing the interface (having preference over the default implementation). So this enables us to mimic multiple class extension in this aspect.

If a class implements several interfaces and the method is implemented in more than one, the implementation of the most specific one is used (if a class C implements A and B, and A implements B also, the method from A is used)

# 2. Lambdas

These expressions allow **functional programming** in Java, avoiding innner classes, and enabling **high level functions** (taking another functions as parameter or returning them).

## High level functions

Example: word collection sorting, using a lambda function as comparator.

In [20]:
List<String> names = Arrays.asList("shekhar", "rahul", "sameer");
Collections.sort(names, (first, second) -> first.length() - second.length());
names;

[rahul, sameer, shekhar]

In [21]:
List<String> names = Arrays.asList("aaa", "bbb", "ccc");
Collections.sort(names, (first, second) -> first.compareTo(second));
names;

[aaa, bbb, ccc]

What we passed as parameter is equivalent to:
```java
Comparator<String> comparator = (first, second) -> first.length() - second.length();
```
The expression corresponds with the compare() method:
```java
int compare(T o1, T o2);
```
We can use this way only interfaces with a single abstract method. We can use the `@FunctionaInterface` tag in front of the interface.

## Built-in functional interfaces

 Java 8 provides a lot of functional interfaces in the `java.util.function` package:

### java.util.function.Predicate
Para comprobación de un predicado (devuelve un booleano).
Ejemplo:
```java
Predicate<String> namesStartingWithS = name -> name.startsWith("s");
```

### java.util.function.Consumer
Para realizar acciones que no devuelven nada.
Ejemplo:
```java
Consumer<String> messageConsumer = message -> System.out.println(message);
```

### java.util.function.Function<T,R>
Para realizar acciones que devuelven resultado.
Ejemplo:
```java
Function<String, String> toUpperCase = name -> name.toUpperCase();
```
Que como simplemente llama a una función podría simplificarse así:
```java
Function<String, Integer> toUpperCase = String::toUpperCase;
```

Otro ejemplo de referencia a método con parámetros:

In [22]:
Function<List<Integer>, Integer> maxFn = Collections::max;
int m = maxFn.apply(Arrays.asList(1, 10, 3, 5));
m;

10

### java.util.function.Supplier
Para generar un resultado sin ninguna entrada.
Ejemplo:
```java
Supplier<String> uuidGenerator= () -> UUID.randomUUID().toString();
```

## Lambdas refactoring

In [23]:
List<String> names = Arrays.asList("shekhar", "rahul", "sameer");

In [24]:
names.forEach(System.out::println)

shekhar
rahul
sameer


In [25]:
Predicate<String> startsWithS = name -> name.startsWith("s");
for (String name : names) {
    if (startsWithS.test(name))
        System.out.println(name);
}

shekhar
sameer


# 3. Streams

Streams replace Collections for data processing. A stream is an abstract view for a data sequence, which is lazy (doesn't hold data until they're executed):

In [26]:
Stream<String> uuidStream = Stream.generate(() -> UUID.randomUUID().toString());
uuidStream;

java.util.stream.ReferencePipeline$Head@3c93f4fc

The Stream API provides operations of 2 types:
- partial: `filter`, `map`, `sorted`
- final: `collect`, `forEach`, `count`

A pipeline example:

In [27]:
List<String> names = Arrays.asList("shekhar", "rahul", "sameer", "sameer");

List<String> readingNames = names.stream()              // creates a pipeline
        .distinct()                                     // takes unique values
        .filter(name -> name.startsWith("s"))           // filters with a predicate
        .sorted((t1, t2) -> t1.length() - t2.length())  // sorts with a comparator
        .map(String::toUpperCase)                       // applies a function
        .collect(Collectors.toList());                  // creates a list with the results

readingNames.forEach(System.out::println);

SAMEER
SHEKHAR


In [28]:
List<String> names = Arrays.asList("shekhar", "rahul", "sameer");

List<String> readingNames = names.stream()
        .filter(name -> name.startsWith("s"))
        .sorted(Comparator.comparing(String::length).reversed())   // Refactor + Reverse order
        .map(String::toUpperCase)
        .collect(Collectors.toList());

readingNames.forEach(System.out::println);

SHEKHAR
SAMEER


Other Stream methods:
- `skip()`
- `limit()`
- `allMatch(<condition>)` -> boolean
- `anyMatch(<condition>)` -> boolean
- `noneMatch(<condition>)` -> boolean
- `findFirst(<condition>)` -> ?
- `findAny(<condition>)` -> ?
- `reduce` ->

Examples:

In [29]:
List<String> names = Arrays.asList("shekhar", "rahul", "sameer");

List<String> readingNames = names.stream()
        .skip(1)                            // skips results (useful for pagination!)
        .limit(2)                           // limits the number of results
        .collect(Collectors.toList());

readingNames.forEach(System.out::println);

rahul
sameer


In [30]:
List<String> names = Arrays.asList("shekhar", "rahul", "sameer");

boolean anyMatch = names.stream()
        .anyMatch(name -> name.equals("rahul"));

anyMatch

true

In [31]:
List<String> names = Arrays.asList("shekhar", "rahul", "sameer");

Optional<String> first = names.stream()
        .findFirst();

first

Optional[shekhar]

In [32]:
List<String> names = Arrays.asList("shekhar", "rahul", "sameer");

String readingNames = names.stream()
        .map(String::toUpperCase)
        .reduce((first, second) -> first + " | " + second)
        .get();

readingNames;

SHEKHAR | RAHUL | SAMEER

## Streams elementales

In [33]:
IntStream.range(0, 10).forEach(System.out::print);

0123456789

In [34]:
IntStream.rangeClosed(0, 10).forEach(System.out::printf());

CompilationException: 

In [35]:
LongStream infiniteStream = LongStream.iterate(1, el -> el + 1);
infiniteStream.filter(el -> el % 2 == 0).limit(5).forEach(System.out::print);

246810

In [36]:
String[] tags = {"java", "git", "lambdas", "machine-learning"};
Arrays.stream(tags).map(String::toUpperCase).forEach(System.out::println);

JAVA
GIT
LAMBDAS
MACHINE-LEARNING


## Flujos paralelos

In [37]:
Map<String, List<Integer>> numbersPerThread = IntStream.rangeClosed(1, 22)
        .parallel()
        .boxed()
        .collect(Collectors.groupingBy(i -> Thread.currentThread().getName()));

numbersPerThread.forEach((k, v) -> System.out.println(String.format("%s >> %s", k, v)));

IJava-executor-0 >> [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
ForkJoinPool.commonPool-worker-3 >> [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [38]:
//String[] urls = {"https://www.google.co.in/", "https://twitter.com/", "http://www.facebook.com/"};
//Arrays.stream(urls).parallel().map(url -> getUrlContent(url)).forEach(System.out::println);

# 4. Collectors

Used for several purposes:
- reduce a stream into a single value
- group elements in a stream
- divide elements in a stream

See the [DOCS](https://docs.oracle.com/javase/8/docs/api/java/util/stream/Collectors.html) for all the available methods.

## Previous code

In [39]:
import java.time.*;
import java.util.*;

public enum TaskType {
    READING, CODING, BLOGGING, WRITING
}

public class Task {

    private final String id;
    private final String title;
    private final String description;
    private final TaskType type;
    private LocalDate createdOn;
    private Set<String> tags = new HashSet<>();

    public Task(final String id, final String title, final TaskType type) {
        this.id = id;
        this.title = title;
        this.description = title;
        this.type = type;
        this.createdOn = LocalDate.now();
    }

    public Task(final String title, final TaskType type) {
        this(title, title, type, LocalDate.now());
    }

    public Task(final String title, final TaskType type, final LocalDate createdOn) {
        this(title, title, type, createdOn);
    }

    public Task(final String title, final String description, final TaskType type, final LocalDate createdOn) {
        this.id = UUID.randomUUID().toString();
        this.title = title;
        this.description = description;
        this.type = type;
        this.createdOn = createdOn;
    }

    public String getId() {
        return id;
    }

    public String getTitle() {
        return title;
    }

    public String getDescription() {
        return description;
    }

    public TaskType getType() {
        return type;
    }

    public LocalDate getCreatedOn() {
        return createdOn;
    }

    public Task addTag(String tag) {
        this.tags.add(tag);
        return this;
    }

    public Set<String> getTags() {
        return Collections.unmodifiableSet(tags);
    }

    @Override
    public String toString() {
        return "Task{" +
                "title='" + title + '\'' +
                ", type=" + type +
                '}';
    }

    @Override
    public boolean equals(Object o) {
        if (this == o) return true;
        if (o == null || getClass() != o.getClass()) return false;
        Task task = (Task) o;
        return Objects.equals(title, task.title) &&
                Objects.equals(type, task.type);
    }

    @Override
    public int hashCode() {
        return Objects.hash(title, type);
    }
}

public class TaskTag {
    final String tag;
    final Task task;

    public TaskTag(String tag, Task task) {
        this.tag = tag;
        this.task = task;
    }

    public String getTag() {
        return tag;
    }

    public Task getTask() {
        return task;
    }
}

Task task1 = new Task("Read Version Control with Git book", TaskType.READING, 
        LocalDate.of(2015, Month.JULY, 1)).addTag("git").addTag("reading").addTag("books");
Task task2 = new Task("Read Java 8 Lambdas book", TaskType.READING, 
        LocalDate.of(2015, Month.JULY, 2)).addTag("java8").addTag("reading").addTag("books");
Task task3 = new Task("Write a mobile application to store my tasks", TaskType.CODING, 
        LocalDate.of(2015, Month.JULY, 3)).addTag("coding").addTag("mobile");
Task task4 = new Task("Write a blog on Java 8 Streams", TaskType.WRITING, 
        LocalDate.of(2015, Month.JULY, 4)).addTag("blogging").addTag("writing").addTag("streams");
Task task5 = new Task("Read Domain Driven Design book", TaskType.READING, 
        LocalDate.of(2015, Month.JULY, 5)).addTag("ddd").addTag("books").addTag("reading");

List<Task> tasks = Arrays.asList(task1, task2, task3, task4, task5);

## Generating a value

To list, to set:

In [40]:
tasks.stream().map(Task::getTitle).collect(Collectors.toList());

[Read Version Control with Git book, Read Java 8 Lambdas book, Write a mobile application to store my tasks, Write a blog on Java 8 Streams, Read Domain Driven Design book]

In [41]:
tasks.stream().map(Task::getTitle).collect(Collectors.toSet());

[Write a blog on Java 8 Streams, Read Domain Driven Design book, Read Java 8 Lambdas book, Write a mobile application to store my tasks, Read Version Control with Git book]

To map:

In [42]:
tasks.stream().collect(Collectors.toMap(Task::getTitle, Function.identity())); // identity() = task -> task

{Write a blog on Java 8 Streams=Task{title='Write a blog on Java 8 Streams', type=WRITING}, Read Domain Driven Design book=Task{title='Read Domain Driven Design book', type=READING}, Read Java 8 Lambdas book=Task{title='Read Java 8 Lambdas book', type=READING}, Write a mobile application to store my tasks=Task{title='Write a mobile application to store my tasks', type=CODING}, Read Version Control with Git book=Task{title='Read Version Control with Git book', type=READING}}

To map, with duplicates control:

In [43]:
tasks.stream().collect(Collectors.toMap(Task::getType, Function.identity(), (t1, t2) -> t2));

{WRITING=Task{title='Write a blog on Java 8 Streams', type=WRITING}, READING=Task{title='Read Domain Driven Design book', type=READING}, CODING=Task{title='Write a mobile application to store my tasks', type=CODING}}

To another type of map:

In [44]:
tasks.stream().collect(Collectors.toMap(Task::getType, Function.identity(), (t1, t2) -> t2, LinkedHashMap::new));

{READING=Task{title='Read Domain Driven Design book', type=READING}, CODING=Task{title='Write a mobile application to store my tasks', type=CODING}, WRITING=Task{title='Write a blog on Java 8 Streams', type=WRITING}}

To other types:

In [45]:
tasks.stream().collect(Collectors.toCollection(LinkedHashSet::new));

[Task{title='Read Version Control with Git book', type=READING}, Task{title='Read Java 8 Lambdas book', type=READING}, Task{title='Write a mobile application to store my tasks', type=CODING}, Task{title='Write a blog on Java 8 Streams', type=WRITING}, Task{title='Read Domain Driven Design book', type=READING}]

Find the longest title:

In [46]:
tasks.stream().collect(Collectors.collectingAndThen(Collectors.maxBy(
        (t1, t2) -> t1.getTitle().length() - t2.getTitle().length()), Optional::get));

Task{title='Write a mobile application to store my tasks', type=CODING}

Count tag number:

In [47]:
tasks.stream().collect(Collectors.summingInt(task -> task.getTags().size()));

14

Titles summary:

In [48]:
tasks.stream().map(Task::getTitle).collect(Collectors.joining("; "));

Read Version Control with Git book; Read Java 8 Lambdas book; Write a mobile application to store my tasks; Write a blog on Java 8 Streams; Read Domain Driven Design book

## Grouping accumulators

Group by type:

In [49]:
tasks.stream().collect(Collectors.groupingBy(Task::getType));

{WRITING=[Task{title='Write a blog on Java 8 Streams', type=WRITING}], READING=[Task{title='Read Version Control with Git book', type=READING}, Task{title='Read Java 8 Lambdas book', type=READING}, Task{title='Read Domain Driven Design book', type=READING}], CODING=[Task{title='Write a mobile application to store my tasks', type=CODING}]}

Group by tag:

In [50]:
tasks.stream().flatMap(task -> task.getTags().stream().map(tag -> new TaskTag(tag, task))).
        collect(Collectors.groupingBy(TaskTag::getTag, Collectors.mapping(TaskTag::getTask, Collectors.toList())));

{coding=[Task{title='Write a mobile application to store my tasks', type=CODING}], books=[Task{title='Read Version Control with Git book', type=READING}, Task{title='Read Java 8 Lambdas book', type=READING}, Task{title='Read Domain Driven Design book', type=READING}], git=[Task{title='Read Version Control with Git book', type=READING}], blogging=[Task{title='Write a blog on Java 8 Streams', type=WRITING}], ddd=[Task{title='Read Domain Driven Design book', type=READING}], java8=[Task{title='Read Java 8 Lambdas book', type=READING}], streams=[Task{title='Write a blog on Java 8 Streams', type=WRITING}], writing=[Task{title='Write a blog on Java 8 Streams', type=WRITING}], mobile=[Task{title='Write a mobile application to store my tasks', type=CODING}], reading=[Task{title='Read Version Control with Git book', type=READING}, Task{title='Read Java 8 Lambdas book', type=READING}, Task{title='Read Domain Driven Design book', type=READING}]}

Group by task and date:

In [51]:
tasks.stream().collect(Collectors.groupingBy(Task::getType, Collectors.groupingBy(Task::getCreatedOn)));

{WRITING={2015-07-04=[Task{title='Write a blog on Java 8 Streams', type=WRITING}]}, READING={2015-07-05=[Task{title='Read Domain Driven Design book', type=READING}], 2015-07-02=[Task{title='Read Java 8 Lambdas book', type=READING}], 2015-07-01=[Task{title='Read Version Control with Git book', type=READING}]}, CODING={2015-07-03=[Task{title='Write a mobile application to store my tasks', type=CODING}]}}

## Dividing

Division made on a date:

In [52]:
tasks.stream().collect(Collectors.partitioningBy(task -> task.getCreatedOn().isAfter(LocalDate.now())));

{false=[Task{title='Read Version Control with Git book', type=READING}, Task{title='Read Java 8 Lambdas book', type=READING}, Task{title='Write a mobile application to store my tasks', type=CODING}, Task{title='Write a blog on Java 8 Streams', type=WRITING}, Task{title='Read Domain Driven Design book', type=READING}], true=[]}

# 5. Optionals

We can see an Optional like a single element Stream. It has methods similar to Stream API like `map`, `filter`, and `flatMap`, which we can use to work with values contained in the Optional.

There are 3 ways of creating them:

- `Optional.empty`: used to create an Optional when a value is not present.
- `Optional.of(T value)`: used to create an Optional from a non-null value. It throws NullPointerException when value is null.
- **`Optional.ofNullable(T value)`**: static factory method which works for both null and non-null values. For null values it will create an empty Optional and for non-null values it will create an Optional using the value.

Example with **`map`**:

In [53]:
public String taskTitle(String taskId) {
    return taskRepository.
            find(taskId).
            map(Task::getTitle).    // Translates from Optional<Task> to Optional<String>
            orElseThrow(() -> new TaskNotFoundException(String.format("No task exist for id '%s'",taskId)));
}

There are 3 different `orElse*` methods:

- `orElse(T t)`: used to return the value if it exists, or returns the value t passed as parameter, like Optional.ofNullable(null).orElse("NoValue"). This will return "NoValue" as no value exists.
- `orElseGet()`: used to return the value if it exists, otherwise invokes the Supplier's get method to produce a new value. For example, Optional.ofNullable(uuid).orElseGet(() -> UUID.randomUUID().toString() could be used to lazily produce a value only when none is present.
- `orElseThrow()`: this allow clients to throw their own custom exception when a value is not present.

Example with **`flatMap`**:

In [54]:
public String taskAssignedTo(String taskId) {
    return taskRepository.
            find(taskId).
            flatMap(task -> task.getAssignedTo().map(user -> user.getUsername())).
            orElse("NotAssigned");
}

Example with **`filter`**:

In [55]:
public boolean isTaskDueToday(Optional<Task> task) {
    return task.flatMap(Task::getCreatedOn).filter(d -> d.isEqual(LocalDate.now())).isPresent();
}

# 6. Map improvements

Create a Map from a List (we could create more types of Map):

In [56]:
 Map<String, Task> taskIdToTaskMap = tasks.stream()
         .collect(Collectors.toMap(Task::getId, Function.identity()));
 taskIdToTaskMap

{02b4597b-6d91-4b9e-8b01-a4b98deb1a65=Task{title='Write a mobile application to store my tasks', type=CODING}, f5be10c3-dc81-465d-9f05-f51c95c2aeb8=Task{title='Read Version Control with Git book', type=READING}, 8b34906b-e53b-4ad5-be57-d94821cb05af=Task{title='Read Domain Driven Design book', type=READING}, b235a4c5-11b5-4295-9c24-e56e9426c737=Task{title='Write a blog on Java 8 Streams', type=WRITING}, 8133a563-b529-4e69-bdf1-2e4e2c090008=Task{title='Read Java 8 Lambdas book', type=READING}}

Removing duplicates:

In [57]:
Map<String, Task> taskIdToTaskMap2 = tasks.stream()
        .collect(Collectors.toMap(Task::getId, Function.identity(), (k1, k2) -> k2));
taskIdToTaskMap2

{02b4597b-6d91-4b9e-8b01-a4b98deb1a65=Task{title='Write a mobile application to store my tasks', type=CODING}, f5be10c3-dc81-465d-9f05-f51c95c2aeb8=Task{title='Read Version Control with Git book', type=READING}, 8b34906b-e53b-4ad5-be57-d94821cb05af=Task{title='Read Domain Driven Design book', type=READING}, b235a4c5-11b5-4295-9c24-e56e9426c737=Task{title='Write a blog on Java 8 Streams', type=WRITING}, 8133a563-b529-4e69-bdf1-2e4e2c090008=Task{title='Read Java 8 Lambdas book', type=READING}}

We can also create a Map from tuples.

# 7. Date Time API

The 3 classes that we'll encounter most in the new API are:

- `LocalDate`: represents a date with no time or timezone.
- `LocalTime`: represents time with no date or timezone.
- `LocalDateTime`: is the combination of LocalDate and LocalTime (date with time without time zone).

In [58]:
import java.time.*;
import java.time.format.*;
import java.time.temporal.*;

## LocalDate

In [59]:
LocalDate.of(2019, Month.JULY, 8);

2019-07-08

In [60]:
LocalDate.ofYearDay(2015, 90)

2015-03-31

In [61]:
LocalDate.ofEpochDay(1)

1970-01-02

In [62]:
LocalDate ld = LocalDate.now();
ld

2019-07-08

In [63]:
System.out.print(ld.getMonth() + ", " + ld.getYear() + ", " + ld.getDayOfMonth() + ", " + ld.getDayOfYear())

JULY, 2019, 8, 189

## LocalTime

In [64]:
LocalTime.of(1, 15);

01:15

In [65]:
LocalTime lt = LocalTime.now();
lt;

15:01:40.579682

In [66]:
System.out.print(lt.getHour() + ", " + lt.getMinute() + ", " + lt.getSecond())

15, 1, 40

## LocalDateTime

In [67]:
LocalDateTime.of(ld, lt);

2019-07-08T15:01:40.579682

In [68]:
LocalDateTime ldt = LocalDateTime.now();
ldt

2019-07-08T15:01:40.890058

## Manipulating dates

In [69]:
ld

2019-07-08

In [70]:
ld.plusDays(1);

2019-07-09

In [71]:
ld.plusWeeks(1);

2019-07-15

In [72]:
ld.plusMonths(1);

2019-08-08

In [73]:
ld.plusYears(1);

2020-07-08

In [74]:
ld.minusYears(1);

2018-07-08

In [75]:
lt

15:01:40.579682

In [76]:
lt.plusSeconds(1)

15:01:41.579682

In [77]:
lt.plusMinutes(1)

15:02:40.579682

In [78]:
lt.plusHours(1)

16:01:40.579682

In [79]:
lt.minusHours(1)

14:01:40.579682

### Duration

In [80]:
Duration.between(LocalTime.of(1, 0), lt)

PT14H1M40.579682S

### Period

In [81]:
Period.between(LocalDate.of(1969, Month.JULY, 20), ld);

P49Y11M18D

### Formatting and Parsing

In [82]:
DateTimeFormatter formatter = DateTimeFormatter.ofPattern("dd-MM-YYYY - HH:mm:ss");
ldt.format(formatter);

08-07-2019 - 15:01:40

In [83]:
LocalDateTime.parse("15 Oct 1931 01:15 AM", DateTimeFormatter.ofPattern("dd MMM yyyy hh:mm a"));

1931-10-15T01:15

### Temporal Adjusters

In [84]:
ldt.with(TemporalAdjusters.firstDayOfMonth());

2019-07-01T15:01:40.890058

In [85]:
ldt.with(TemporalAdjusters.firstDayOfNextMonth());

2019-08-01T15:01:40.890058

In [86]:
ldt.with(TemporalAdjusters.lastInMonth(DayOfWeek.FRIDAY));

2019-07-26T15:01:40.890058

See the [DOCS](https://docs.oracle.com/javase/8/docs/api/java/time/temporal/TemporalAdjusters.html) for all the temporal adjusters.